In [1]:
import numpy as np
from caffe2.python import core, model_helper, workspace, utils
from game import InitGame, AddGamePlay
from modeling import AddConvModel
mini_batch=3

In [2]:
import caffe2.python.predictor.predictor_exporter as pe
import os
from caffe2.proto import caffe2_pb2

# load the predict net
LOAD_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_data','go','param','RL-conv=13-k=192-iter=6')
predict_net = pe.prepare_prediction_net(os.path.join(LOAD_FOLDER, "policy_model.minidb"), "minidb", device_option=core.DeviceOption(caffe2_pb2.CPU, 0)
)
print('Params loaded from {}'.format(LOAD_FOLDER))

Params loaded from /home/wangd/python/tutorial_data/go/param/RL-conv=13-k=192-iter=6


## Init first step

In [3]:
ZERO = np.zeros((mini_batch,1,19,19), dtype=np.float32)
ONE = np.ones((mini_batch,1,19,19), dtype=np.float32)
init_data = np.concatenate((ZERO,ZERO,ONE,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,
                            ZERO,ZERO,ZERO,ZERO), axis=1)
workspace.FeedBlob("data", init_data)

model = model_helper.ModelHelper(name="model", arg_scope={"order": "NCHW"}, init_params=False)
AddConvModel(model, "data", dim_in=48)
AddGamePlay(model, "data", "predict")

workspace.RunNetOnce(model.param_init_net)
workspace.CreateNet(model.net, overwrite=True)
workspace.RunNet(model.net)

init_move = np.reshape(workspace.FetchBlob('predict')[0], (-1)) # shape=(361,)
top_choice = np.argsort(-init_move)[0:mini_batch] # the top K step

for i in range(mini_batch):
    x = top_choice[i]/19
    y = top_choice[i]%19
    print('({},{})'.format(x,y))
    init_data[i,1,x,y] = 1 # opponent plus (x,y)
    init_data[i,2,x,y] = 0 # empty minus (x,y)
    init_data[i,4,x,y] = 1 # last 1 step plus (x,y)
    #init_data[i,12] = -1

workspace.FeedBlob("data", init_data)

(4,2)
(5,2)
(4,4)


True

In [4]:
for i in range(20):
    workspace.RunNet(model.net)
    label=workspace.FetchBlob("label")
    label = [ (i/19,i%19) for i in label]
    print(label)

[(1, 4), (1, 4), (16, 16)]
[(6, 2), (7, 2), (3, 5)]
[(2, 3), (2, 3), (4, 12)]
[(16, 16), (4, 1), (2, 2)]
[(3, 8), (2, 7), (16, 14)]
[(2, 16), (2, 16), (2, 16)]
[(5, 16), (4, 16), (4, 10)]
[(1, 14), (5, 15), (4, 2)]
[(2, 7), (4, 15), (1, 9)]
[(1, 11), (1, 14), (4, 16)]
[(4, 4), (7, 15), (1, 6)]
[(4, 15), (1, 11), (1, 14)]
[(5, 3), (2, 2), (1, 4)]
[(1, 9), (1, 8), (6, 4)]
[(5, 2), (5, 16), (2, 8)]
[(5, 1), (1, 6), (6, 16)]
[(4, 17), (4, 3), (1, 3)]
[(1, 8), (2, 10), (2, 13)]
[(2, 17), (6, 3), (9, 16)]
[(1, 7), (5, 14), (1, 11)]


In [5]:
data = workspace.FetchBlob('data')
np.array(data[2,2],dtype=int)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1,